# Initialize Env

In [1]:
from my_utils import init_env
import mediapy as media
env = init_env()

[robosuite WARNING] No private macro file found! (macros.py:53)
[robosuite WARNING] It is recommended to use a private macro file (macros.py:54)
[robosuite WARNING] To setup, run: python /research/rs4tmr/robosuite/scripts/setup_macros.py (macros.py:55)


Robosuite environment maked: <class 'robosuite.environments.manipulation.tmr_pp.TmrPickPlaceCan'> <robosuite.environments.manipulation.tmr_pp.TmrPickPlaceCan object at 0x7f147cf821f0> ['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_action_dim', '_check_grasp', '_check_robot_configuration', '_check_success', '_construct_objects', '_construct_visual_objects', '_create_camera_sensors', '_create_obj_sensors', '_create_segementation_sensor', '_destroy_sim', '_destroy_viewer', '_eef_xmat', '_eef_xpos', '_eef_xquat', '_get_observations', '_get_placement_initializer', '_gripper_to_target', '_initialize_sim', '_input2list', '_load_model', '_load_robots', '_obs_cache', '_observables', '_post_action', '

KeyError: 'agentview_image'

In [2]:
obs = env.reset()
print(obs[0].shape)
#env.step(env.action_space.sample())

Observation Setup
odict_keys(['robot0_joint_pos_cos', 'robot0_joint_pos_sin', 'robot0_joint_vel', 'robot0_eef_pos', 'robot0_eef_quat', 'robot0_gripper_qpos', 'robot0_gripper_qvel', 'Can_pos', 'Can_quat', 'Can_to_robot0_eef_pos', 'Can_to_robot0_eef_quat', 'robot0_proprio-state', 'object-state'])
(102,)


# Get images

In [3]:
env.image_states

/root/anaconda3/envs/rs4tmr/lib/python3.9/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.image_states to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.image_states` for environment variables or `env.get_wrapper_attr('image_states')` that will search the reminding wrappers.
  logger.warn(


{'agentview_image': array([[[203, 190, 182],
         [211, 198, 189],
         [208, 196, 185],
         ...,
         [186, 177, 168],
         [191, 182, 173],
         [187, 178, 169]],
 
        [[203, 190, 182],
         [208, 195, 186],
         [210, 197, 187],
         ...,
         [191, 182, 173],
         [193, 184, 175],
         [179, 170, 161]],
 
        [[205, 193, 185],
         [205, 192, 184],
         [208, 195, 186],
         ...,
         [203, 195, 186],
         [191, 183, 174],
         [184, 175, 166]],
 
        ...,
 
        [[117, 115, 111],
         [118, 116, 112],
         [118, 116, 112],
         ...,
         [117, 115, 111],
         [117, 115, 111],
         [116, 114, 111]],
 
        [[118, 117, 113],
         [118, 116, 113],
         [118, 116, 113],
         ...,
         [118, 116, 112],
         [117, 116, 112],
         [117, 115, 112]],
 
        [[118, 116, 113],
         [118, 116, 112],
         [117, 115, 112],
         ...,
         

# Get observation according key name

In [4]:
env.reset()
obs = env.step(env.action_space.sample())
env.get_obs_according_to_key(obs[0],'robot0_joint_pos_cos')

/root/anaconda3/envs/rs4tmr/lib/python3.9/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.get_obs_according_to_key to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.get_obs_according_to_key` for environment variables or `env.get_wrapper_attr('get_obs_according_to_key')` that will search the reminding wrappers.
  logger.warn(


array([ 0.89218116, -0.19680942, -0.77927584, -0.66216597, -0.02516486,
       -0.41917018])

# Model Checkpoint load and Rollout

In [1]:
"""
"""

from stable_baselines3 import SAC
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv
from my_utils import init_env

#from my_utils import make_env
import mediapy as media


env_id = 'TmrPickPlaceCan'
horizon = 1000
frames = []

print('making env')
vec_env = DummyVecEnv([init_env])

print('load model')
model = SAC.load(env_id, env=vec_env)

print('reset env')
obs = vec_env.reset()

print('_observables: ')
for key, val in vec_env.envs[0].env.env._observables.items():
    print(key, val)
    pass
for t in range(horizon):
    #print('predict by model')
    action, _states = model.predict(obs)

    #print('step forward')
    obs, rewards, dones, info = vec_env.step(action)
    #print(t, len(obs), obs)
    
    #print('rendering env')
    rendered_array=vec_env.envs[0].sim.render(width=640, height=480, camera_name='frontview', device_id=0) # 'frontview', 'birdview', 'agentview', 'robot0_robotview', 'robot0_eye_in_hand')
    # revese the image with respect to x-axis
    rendered_array = rendered_array[::-1, :, :]
    
    frames.append(rendered_array)
media.show_video(frames)

[robosuite WARNING] No private macro file found! (macros.py:53)
[robosuite WARNING] It is recommended to use a private macro file (macros.py:54)
[robosuite WARNING] To setup, run: python /research/rs4tmr/robosuite/scripts/setup_macros.py (macros.py:55)


making env
Initalized env with init_env
Selected observables: ['robot0_joint_pos', 'robot0_joint_vel', 'robot0_eef_pos', 'robot0_eef_quat', 'robot0_eef_vel_lin', 'robot0_eef_vel_ang', 'robot0_gripper_qpos', 'robot0_gripper_qvel', 'Can_to_robot0_eef_pos', 'Can_to_robot0_eef_quat']
Robosuite environment maked: <class 'robosuite.environments.manipulation.tmr_pp.TmrPickPlaceCan'> <robosuite.environments.manipulation.tmr_pp.TmrPickPlaceCan object at 0x7f996b54d0d0> ['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_action_dim', '_check_grasp', '_check_robot_configuration', '_check_success', '_construct_objects', '_construct_visual_objects', '_create_camera_sensors', '_create_obj_sensors', '_create_se

Error: You must call wandb.init() before wandb.log()

In [30]:
obs = vec_env.reset()
print(obs.shape)

(1, 51)


In [1]:
import robosuite as suite
from robosuite.wrappers import GymWrapper

from stable_baselines3.common.utils import set_random_seed
from stable_baselines3.common.monitor import Monitor


rsenv = suite.make(
    "TmrPickPlaceCan",
    robots="UR5e",  # use UR5e robot
    use_camera_obs=False,  # use pixel observations
    has_offscreen_renderer=True,  # needed if using pixel obs
    has_renderer=True,  # make sure we can render to the screen
    reward_shaping=True,  # use dense rewards
    control_freq=20,  # control should happen fast enough so that simulation looks smooth
)

# full_observable_list = [
#     'robot0_joint_pos', 
#     'robot0_joint_pos_cos', 
#     'robot0_joint_pos_sin', 
#     'robot0_joint_vel', 
#     'robot0_eef_pos', 
#     'robot0_eef_quat', 
#     'robot0_eef_vel_lin', 
#     'robot0_eef_vel_ang', 
#     'robot0_gripper_qpos', 
#     'robot0_gripper_qvel', 
#     'agentview_image', 
#     'world_pose_in_gripper', 
#     'Milk_pos', 
#     'Milk_quat', 
#     'Milk_to_robot0_eef_pos', 
#     'Milk_to_robot0_eef_quat', 
#     'Bread_pos', 
#     'Bread_quat', 
#     'Bread_to_robot0_eef_pos', 
#     'Bread_to_robot0_eef_quat', 
#     'Cereal_pos', 
#     'Cereal_quat', 
#     'Cereal_to_robot0_eef_pos', 
#     'Cereal_to_robot0_eef_quat', 
#     'Can_pos', 
#     'Can_quat', 
#     'Can_to_robot0_eef_pos', 
#     'Can_to_robot0_eef_quat',
# ]
# for observable in full_observable_list:
#     rsenv.modify_observable(observable, 'enabled', True)
#     rsenv.modify_observable(observable, 'active', True)

useless_observable_list = [
    # 'robot0_joint_pos', 
    # 'robot0_joint_pos_cos', 
    # 'robot0_joint_pos_sin', 
    # 'robot0_joint_vel', 
    # 'robot0_eef_pos', 
    # 'robot0_eef_quat', 
    # 'robot0_eef_vel_lin', 
    # 'robot0_eef_vel_ang', 
    # 'robot0_gripper_qpos', 
    # 'robot0_gripper_qvel', 
    # 'agentview_image', 
    # 'world_pose_in_gripper', 
    # 'Milk_pos', 
    # 'Milk_quat', 
    # 'Milk_to_robot0_eef_pos', 
    # 'Milk_to_robot0_eef_quat', 
    # 'Bread_pos', 
    # 'Bread_quat', 
    # 'Bread_to_robot0_eef_pos', 
    # 'Bread_to_robot0_eef_quat', 
    # 'Cereal_pos', 
    # 'Cereal_quat', 
    # 'Cereal_to_robot0_eef_pos', 
    # 'Cereal_to_robot0_eef_quat', 
    # 'Can_pos', 
    # 'Can_quat', 
    # 'Can_to_robot0_eef_pos', 
    # 'Can_to_robot0_eef_quat',
]
for observable in useless_observable_list:
    rsenv.modify_observable(observable, 'enabled', False)
    rsenv.modify_observable(observable, 'active', False)

print('Robosuite environment maked:',type(rsenv) , rsenv, dir(rsenv))
print(len(rsenv._observables.keys()))
print(rsenv._observables.keys())

env = GymWrapper(
    rsenv
)

env.reset(seed=0)

new_env = Monitor(env)


[robosuite WARNING] No private macro file found! (macros.py:53)
[robosuite WARNING] It is recommended to use a private macro file (macros.py:54)
[robosuite WARNING] To setup, run: python /research/rs4tmr/robosuite/scripts/setup_macros.py (macros.py:55)


Robosuite environment maked: <class 'robosuite.environments.manipulation.tmr_pp.TmrPickPlaceCan'> <robosuite.environments.manipulation.tmr_pp.TmrPickPlaceCan object at 0x7ff04aaedeb0> ['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_action_dim', '_check_grasp', '_check_robot_configuration', '_check_success', '_construct_objects', '_construct_visual_objects', '_create_camera_sensors', '_create_obj_sensors', '_create_segementation_sensor', '_destroy_sim', '_destroy_viewer', '_eef_xmat', '_eef_xpos', '_eef_xquat', '_get_observations', '_get_placement_initializer', '_gripper_to_target', '_initialize_sim', '_input2list', '_load_model', '_load_robots', '_obs_cache', '_observables', '_post_action', '

In [18]:
rsenv = suite.make(
    "TmrPickPlaceCan",
    robots="UR5e",  # use UR5e robot
    use_camera_obs=False,  # use pixel observations
    has_offscreen_renderer=True,  # needed if using pixel obs
    has_renderer=True,  # make sure we can render to the screen
    reward_shaping=True,  # use dense rewards
    control_freq=20,  # control should happen fast enough so that simulation looks smooth
)
rsenv.sim.render(width=640, height=480, camera_name='frontview', device_id=0)

array([[[164, 155, 147],
        [162, 153, 145],
        [163, 154, 146],
        ...,
        [179, 170, 163],
        [163, 154, 146],
        [181, 172, 164]],

       [[165, 156, 149],
        [163, 154, 146],
        [162, 153, 145],
        ...,
        [163, 155, 146],
        [177, 169, 160],
        [197, 188, 179]],

       [[167, 158, 151],
        [165, 156, 148],
        [163, 154, 146],
        ...,
        [185, 177, 168],
        [192, 183, 174],
        [189, 180, 171]],

       ...,

       [[129, 127, 123],
        [129, 127, 123],
        [129, 128, 124],
        ...,
        [ 86,  85,  82],
        [ 85,  84,  82],
        [ 86,  84,  82]],

       [[129, 128, 124],
        [129, 127, 123],
        [128, 127, 122],
        ...,
        [ 85,  84,  81],
        [ 85,  84,  81],
        [ 86,  85,  82]],

       [[128, 126, 121],
        [128, 125, 120],
        [127, 124, 120],
        ...,
        [ 84,  82,  79],
        [ 85,  83,  80],
        [ 85,  84,  81]]

In [6]:
print(new_env.render())

: 